In [1]:
import tensorflow as tf
import numpy as np
import cv2, glob, os
from sklearn.model_selection import train_test_split

In [2]:
dataset_path = 'dataset/'
classes = 4
num_epochs = 5
learning_rate = 0.01
width = 128
height = 128
Y = None
Z = None
tf.reset_default_graph()

In [3]:
def load_frames():
    global Z
    global Y
    label = None
    X = []
    filenum = 0
    Y = np.zeros((40,4))
    for file in glob.glob(dataset_path + '**', recursive=True):
        num_frame = 0
        if file == dataset_path:
            continue
        if os.path.isdir(file):
            x_list = []
            label = file[len(dataset_path):] #label   
        if os.path.isfile(file):
            x_list = []
            vidcap = cv2.VideoCapture(file)
            while(vidcap.isOpened()):
                ret, frame = vidcap.read()
                if ret == True:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    each_frame = cv2.resize(frame, (height, width))
                    x_list.append(np.array(each_frame))
                    num_frame +=1
                else:
                    X.append(np.array(x_list))
                    vidcap.release()
            Y[filenum, int(label)] = 1.0
            filenum += 1

   
    Z = np.array(X)

In [4]:
load_frames()

In [5]:
X, X_test, Y, Y_test = train_test_split(Z,Y, test_size=0.0, random_state=30)

In [6]:
X[0].shape

(82, 128, 128)

In [7]:
class RNN:
    
    def vanilla_rnn(self, prev_state, x):
        
        self.U_w = tf.get_variable('U_w', shape=[height*width, 100], dtype=tf.float64)
        self.U_b = tf.get_variable('U_b', shape=[1, 100], dtype=tf.float64)
        
        self.W_w = tf.get_variable('W_w', shape=[200,100],  dtype=tf.float64)
        self.W_b = tf.get_variable('W_b', shape=[1,100],  dtype=tf.float64)
        
        vectorized_input = tf.reshape(x, (1, height*width))
        transform_input = tf.nn.relu(tf.matmul(vectorized_input, self.U_w) + self.U_b)
        state_concat = tf.concat([transform_input, prev_state], 1)
        next_state = tf.nn.relu(tf.matmul(state_concat, self.W_w) + self.W_b)
        
        return next_state
        
    
    def __init__(self):
        
        self.X_placeholder = tf.placeholder(tf.float64, [None, width, height])
        self.Y_placeholder = tf.placeholder(tf.int64, [1, classes])
    
        self.init_state = tf.placeholder(tf.float64, [1,100])
        
        self.O_w = tf.Variable(np.random.rand(100,4), dtype=tf.float64)
        self.O_b = tf.Variable(np.random.rand(1,4), dtype=tf.float64)
        
        self.states = tf.scan(self.vanilla_rnn, self.X_placeholder, initializer=self.init_state, name='states')
        
        
        logit = tf.matmul(self.states[-1], self.O_w) + self.O_b
        self.prediction = tf.nn.softmax(logit)
       
        self.loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,labels=self.Y_placeholder)
        self.train_step = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

In [8]:
with tf.Session() as sess:
    loss = 0.0
    
    model = RNN()
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        
        for num_data in range(Y.shape[0]):
            _current_state = np.zeros((1,100))
            label = np.reshape(Y[num_data], (1,4))
            
            _loss, _train_step, _current_state, _prediction = sess.run(
                [model.loss, model.train_step, model.states, model.prediction],
                feed_dict = {
                    model.X_placeholder:X[num_data],
                    model.Y_placeholder:label,
                    model.init_state:_current_state
                })
            loss += _loss
        print("Step %d | Loss %g"%(epoch, loss))
        loss = 0.0          

Step 0 | Loss 941753
Step 1 | Loss 57.5039
Step 2 | Loss 56.6964
Step 3 | Loss 56.3878
Step 4 | Loss 56.2668
